In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import os
import re

In [2]:
def read_excel_files_from_folder(folder_path):
    
    # Check if the folder exists
    if not os.path.isdir(folder_path):
        print(f"Error: The folder at '{folder_path}' does not exist.")
        return {}

    excel_data = {}

    # Loop through all files in the specified folder
    for filename in os.listdir(folder_path):
        
        # Check if the file is an Excel file
        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)
            
            try:
                # Read the Excel file into a pandas DataFrame
                df = pd.read_excel(file_path)
                excel_data[filename] = df
            except Exception as e:
                print(f"Could not read {filename}: {e}")
                
    return excel_data

data_folder = 'data'
all_excel_dfs = read_excel_files_from_folder(data_folder)

transactions = pd.concat(all_excel_dfs.values())
transactions['Date'] = pd.to_datetime(transactions['Date'], format='%d/%m/%y', errors='coerce')

transactions = transactions.sort_values('Date', ascending= False)


In [6]:

pd.reset_option('all')
transactions[~transactions.Deposit.isna()]

/var/folders/l4/7cp75vs51b3d0tnyvjd4p9fr0000gn/T/ipykernel_2669/1418019656.py:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/var/folders/l4/7cp75vs51b3d0tnyvjd4p9fr0000gn/T/ipykernel_2669/1418019656.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


,Date,Description,Withdrawal,Deposit,Balance
259,2025-07-31,Interest Earned,NaN,0.07,877.67
203,2025-07-11,Salary ACCOUNTANT-GENERAL 05528200563010016400472,NaN,2758.38,3356.66
202,2025-07-10,"Cash Deposit Machine 01816980,TG PAGAR DL3",NaN,160.00,598.28
183,2025-06-30,Interest Earned,NaN,0.05,737.40
155,2025-06-11,Salary ACCOUNTANT-GENERAL 05425200482000015990033,NaN,57.31,1320.07
...,...,...,...,...,...
7,2019-02-28,Interest Earned,NaN,0.02,1005.05
6,2019-02-23,"Cash Deposit Machine 36179415,TG PAGAR BR B",NaN,550.00,1005.03
4,2019-01-31,Interest Earned,NaN,0.01,500.03
2,2019-01-21,Cash,NaN,500.00,510.00


In [ ]:
# Create a dictionary of rules for withdrawals
withdrawal_rules = {
    'Food & Groceries': ['supermarket', 'bakery', 'restaurant', 'cafe', 'food court'],
    'Transport': ['uber', 'grab', 'bus', 'mrt', 'petrol station'],
    'Bills': ['phone bill', 'electricity', 'internet'],
    'Shopping': ['shopee', 'lazada', 'amazon', 'department store'],
    'Health': ['pharmacy', 'clinic', 'hospital']
}


In [31]:
# Create a dictionary of rules for deposits
deposit_rules = {
    'Salary': ['salary'],
    'Cash Deposit': ['cash deposit', 'cash'],
    'Government': ['mindef', 'gov', 'saf', 'gst voucher', 'govt'],
    'Bank Transfer': ['fast payment / receipt', 'funds transfer', 'remittance', 'trfsb', 'fast'],
    'Interest': ['interest'],
    'Refund': ['refund', 'debit card transaction']
}

def categorize_deposit(text, rules):
    """
    Categorizes a deposit description based on a set of rules.

    Args:
        text (str): The deposit description to categorize.
        rules (dict): A dictionary of rules where keys are categories and
                      values are lists of keywords.

    Returns:
        str: The category of the deposit.
    """
    text_lower = text.lower()

    # Rule 1: Check for the 'Other GIRO Deposits' condition first, as it's a
    # special case with a negative constraint.
    if 'payments / collections' in text_lower:
        # Check if any government keywords are present.
        gov_keywords = rules.get('Government', [])
        contains_gov_keyword = any(keyword in text_lower for keyword in gov_keywords)

        # If it contains 'payments / collections' AND does NOT contain any
        # government keywords, it's 'Other GIRO Deposits'.
        if not contains_gov_keyword:
            return 'Other GIRO Deposits'

    # Rule 2: Check for all other defined rules.
    for category, keywords in rules.items():
        # Iterate through the list of keywords for each category.
        for keyword in keywords:
            # We use a regex word boundary (\b) to ensure we match whole words,
            # which can prevent accidental matches in other words.
            if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                return category

    # Fallback: If no rules match, return 'Other'.
    return 'Other'

transactions['Deposit Category'] = np.where(
    transactions['Deposit'].notna(),
    transactions['Description'].apply(lambda x: categorize_deposit(x, deposit_rules)),
    np.nan
)

#transactions['Deposit Category'] = transactions['Description'].apply(lambda x: categorize_deposit(x, deposit_rules))

In [32]:
pd.set_option('display.max_rows', None)
#transactions[~transactions.Deposit.isna()]
#pd.reset_option('all')
transactions[transactions['Deposit Category'] == 'Other']

,Date,Description,Withdrawal,Deposit,Balance,Deposit Category
23,2019-04-08,Advice,NaN,70.02,2222.22,Other


In [ ]:
def categorize_transaction(description, rules):
    """
    Categorizes a single transaction based on a set of rules.

    Args:
        description (str): The description of the transaction.
        rules (dict): A dictionary of categories and their associated keywords.

    Returns:
        str: The assigned category, or 'Uncategorized' if no match is found.
    """
    if pd.isna(description):
        return 'Uncategorized'
        
    # Check for keywords in a case-insensitive manner
    desc_lower = description.lower()
    for category, keywords in rules.items():
        if any(keyword in desc_lower for keyword in keywords):
            return category
            
    return 'Uncategorized'

In [ ]:
# Apply the categorization to create new columns
transactions['Withdrawal_Category'] = transactions.apply(
    lambda row: categorize_transaction(row['descriptions'], withdrawal_rules) if row['withdrawal'] > 0 else 'N/A',
    axis=1
)

transactions['Deposit_Category'] = transactions.apply(
    lambda row: categorize_transaction(row['descriptions'], deposit_rules) if row['deposit'] > 0 else 'N/A',
    axis=1
)

# Display the first few rows with the new categories to verify
print(transactions.head())

In [6]:
total = transactions[transactions['Withdrawal'].isin([90, 100, 110, 120, 130, 140, 150, 160])].sort_values(by = 'Date')
total

,Date,Description,Withdrawal,Deposit,Balance
38,2019-06-08,"Cash Withdrawal 01816980,TG PAGAR BR C",100.0,NaN,3113.16
44,2019-06-29,"Cash Withdrawal 01816980,TG PAGAR BR C",100.0,NaN,3029.48
56,2019-09-07,"Cash Withdrawal 01816980,TG PAGAR BR B",100.0,NaN,3954.82
62,2019-09-21,"Cash Withdrawal 01816980,TG PAGAR BR C",120.0,NaN,3118.62
71,2019-10-18,"Cash Withdrawal 01816980,TG PAGAR BR C",100.0,NaN,3971.82
75,2019-11-01,"Cash Withdrawal 01816980,TG PAGAR MRT 1",100.0,NaN,3801.49
85,2019-12-07,"Cash Withdrawal 01816980,TG PAGAR BR C",100.0,NaN,4604.58
88,2019-12-14,"Cash Withdrawal 01816980,TG PAGAR BR B",100.0,NaN,4487.43
95,2019-12-30,"Cash Withdrawal 01816980,TG PAGAR MRT 2",100.0,NaN,4349.50
96,2019-12-31,"Cash Withdrawal 01816980,TG PAGAR BR C",100.0,NaN,4249.50


In [7]:
total['Withdrawal'].sum()

np.float64(6860.0)